# Create new card set

Taking output from story engine webapp: https://cardcreator.storyenginedeck.com/

and converting it to easier to use csv format for this application

In [17]:
import pandas as pd
import numpy as np
import re
## display options
pd.options.display.max_rows = 4000
pd.options.display.max_columns = 100
pd.set_option('max_colwidth', 100)

In [18]:
### function that splits data into columns
def split_csv_data(df):
    df[['type', 'option1','option2','option3', 'option4']] = df['data'].str.split(';',n=4,expand=True)
    ## remove extra semicolons from last column
    df['option4'] = df['option4'].str.strip(';')
    ## drop the data column
    df.drop(['data'],axis=1, inplace=True)
    return df

### makes dataframe/series for each card type
def make_entry_list(df, val_type):
    # filter dataframe by type
    filtered_df = df[df['type'] == val_type]
    vals = pd.concat([filtered_df['option1'], filtered_df['option2'], filtered_df['option3'], filtered_df['option4']], ignore_index=True)
    vals.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    # vals.dropna(how='any',inplace=True)
    return vals

def fix_agents(agent):
    new_agent = re.sub(r'^\W*\w+\W*', '', agent).capitalize()
    return new_agent

def remove_a(csv_name):
    df = pd.read_csv(f'../data/{csv_name}.csv')
    df['agent'] = df['agent'].apply(fix_agents)
    df.to_csv(fr'../data/{csv_name}.csv', index = False)
    return

def create_cleaned_csv(csv_filename, card_set_name):
    features = ['aspect','agent','engine', 'anchor', 'conflict']
    card_set = pd.DataFrame()
    df = pd.read_csv(f'../data/{csv_filename}', names = ['data'])
    df = split_csv_data(df)
    
    for feature in features:
        card_set[feature] = make_entry_list(df, feature)
        
    card_set.to_csv(f'../data/{card_set_name}.csv', index = False)
    return

### Create Card Sets

In [79]:
create_cleaned_csv('sci_fi.csv', 'sci-fi_cards')
fix_sci_fi()

In [82]:
create_cleaned_csv('politics.csv', 'politics_cards')

In [14]:
create_cleaned_csv('horror.csv', 'horror_cards')

In [19]:
remove_a('horror_cards')

## Testing for Functions

In [87]:
politics = pd.DataFrame()
features = ['agent', 'aspect', 'engine', 'anchor', 'conflict']
### read in data
pol = pd.read_csv('../data/politics.csv', names=['data'])

pol = split_csv_data(pol)

for feature in features:
    politics[feature] = make_entry_list(pol, feature)
    
politics

politics.to_csv(r'../data/politics_cards.csv', index = False)

In [84]:
politics

,aspect,agent,engine,anchor,conflict
0,Government-Funded,Governor,Wants to become the dominant party with,Washington DC,But it will cost them the election
1,Deputized,Mayor,Wants to tip the balance of power with,Big City,But history will see them as a villain
2,Partisan,Insider,Wants to stop a destructive agenda with,A rival nation,But it will end their career
3,Rigged,Analyst,Wants to blow the whistle on/with,A dossier,But it will undermine their own political agenda
4,Poorly Funded,Propagandist,Wants to escape from/with,A top secret program,But they will discover a horrifying truth about their allies
5,Researched,Company Spokesman,Wants to root out corruption with/on,A CIA black site,But bureaucracy will hold them back
6,Highly political,Black Ops Agent,Wants to bribe with/about,An innocent bystander,But they will fail someone relying on them
7,Buried,Fixer,Wants to relinquish control of their party with,Capitol Hill,But the opposing party will come into power
8,State-Funded,Judge,Wants to restore the balance of power with,Small Town,But history will see them as a failure
9,Commissioned,Warlord,Wants to push an aggressive agenda with,An allied nation,But it will end the career of a trusted ally


In [38]:
csv_filename = 'sci_fi.csv'
card_set_name = 'sci-fi_cards'
features = ['agent', 'aspect', 'engine', 'anchor', 'conflict']
card_set = pd.DataFrame()
df = pd.read_csv(f'../data/{csv_filename}', names = ['data'])
df = split_csv_data(df)
# df
# for feature in features:
#     card_set[feature] = make_entry_list(df, feature)

# card_set.to_csv(f'../data/{card_set_name}.csv', index = False)


In [74]:
def create_cleaned_csv(csv_filename, card_set_name):
    features = ['aspect','agent','engine', 'anchor', 'conflict']
    card_set = pd.DataFrame()
    df = pd.read_csv(f'../data/{csv_filename}', names = ['data'])
    df = split_csv_data(df)
    
    for feature in features:
        card_set[feature] = make_entry_list(df, feature)
        
    card_set.to_csv(f'../data/{card_set_name}.csv', index = False)
    return

In [40]:

create_cleaned_csv('sci_fi.csv', 'sci-fi_cards')

### Now read in csv as if we started with it

We need to import CSV, create entry lists with no NaN's, and then create a dict of dataframes to store each card type for use with the other functions we made.

In [30]:
df = pd.read_csv('../data/politics_cards.csv')

In [11]:
# df['anchor']

In [12]:
# df
df_dict = {}
card_types = features
for card_type in card_types:
    values = df[card_type]
    values.dropna(how='any',inplace=True)
    df_dict[card_type] = values

In [26]:
card_set = {}
for card_type in card_types:
    card_set[card_type] = df[card_type].dropna(how='any')

In [28]:
# card_set

In [72]:
def create_card_database(card_set_name):
    ### create dict of cardtypes for given cardset
    df = pd.read_csv(f'../data/{card_set_name}.csv')
    card_set = {}
    card_types = df.columns.to_list()
    for card_type in card_types:
        card_set[card_type] = df[card_type].dropna(how = 'any')
    return card_set

In [77]:
# create_card_database('sci-fi_cards')

## Get rid of A/An in sci-fi list

In [51]:
import re
sf = pd.read_csv('../data/sci-fi_cards.csv')

In [76]:
def fix_agents(agent):
    new_agent = re.sub(r'^\W*\w+\W*', '', agent).capitalize()
    return new_agent

def fix_sci_fi():
    sf = pd.read_csv('../data/sci-fi_cards.csv')
    sf['agent'] = sf['agent'].apply(fix_agents)
    sf.to_csv(r'../data/sci-fi_cards.csv', index = False)
    return

In [54]:
text = sf['agent'].sample().to_string(index = False)

re.sub(r'^\W*\w+\W*', '', text).capitalize()

In [56]:
for agent in sf['agent']:
    text = agent
    re.sub(r'^\W*\w+\W*', '', text).capitalize()

In [61]:
def fix_agents(agent):
    new_agent = re.sub(r'^\W*\w+\W*', '', agent).capitalize()
    return new_agent

sf['agent'] = sf['agent'].apply(fix_agents)

In [63]:
sf.to_csv(r'../data/sci-fi_cards.csv', index = False)

In [66]:
flist = sf.columns.to_list()

In [67]:
flist

## Fixing Horror cards